In [ ]:
from unsloth import FastModel
import torch

dtype = None
load_in_4bit = False

max_seq_length = 2048

model, tokenizer = FastModel.from_pretrained(
    model_name = "OuteAI/Llama-OuteTTS-1.0-1B",
    max_seq_length= max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
    )

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "v_proj",],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
from datasets import load_dataset,Audio,Dataset
dataset = load_dataset("MrDragonFox/Elise", split = "train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=24000))


In [ ]:
#@title Tokenisation Function

import torch
from tqdm import tqdm
import io
import tempfile
from datasets import Dataset
import sys
sys.path.append('OuteTTS')
import os
import dac
# V3 Imports
from outetts.version.v3.audio_processor import AudioProcessor
from outetts.version.v3.prompt_processor import PromptProcessor
from outetts.dac.interface import DacInterface
from outetts.models.config import ModelConfig # Need a dummy config for AudioProcessor
import whisper
from outetts.utils.preprocessing import text_normalizations
import soundfile as sf
import numpy as np

class DataCreationV3:
    def __init__(
            self,
            model_tokenizer_path: str,
            whisper_model_name: str = "turbo",
            device: str = None
        ):

        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Create a dummy ModelConfig mainly for device and paths needed by AudioProcessor/DacInterface
        dummy_config = ModelConfig(
            tokenizer_path=model_tokenizer_path,
            device=self.device,
            audio_codec_path=None # Let AudioProcessor use default DAC path
        )
        self.audio_processor = AudioProcessor(config=dummy_config)
        self.prompt_processor = PromptProcessor(model_tokenizer_path)

        print(f"Loading Whisper model: {whisper_model_name} on {self.device}")
        self.whisper_model = whisper.load_model(whisper_model_name, device=self.device)
        print("Whisper model loaded.")

    # Renamed and adapted from the previous version
    def create_speaker_representation(self, audio_bytes: bytes, transcript: str):
        """
        Creates a v3-compatible speaker dictionary using Whisper and AudioProcessor.
        """
        if not audio_bytes or not transcript:
             print("Missing audio bytes or transcript in create_speaker_representation.")
             return None

        # Whisper needs a file path, so save bytes to a temporary file
        try:
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp_audio_file:
                tmp_audio_file.write(audio_bytes)
                tmp_audio_file.flush() # Ensure data is written

                # 1. Get word timings using Whisper
                whisper_result = self.whisper_model.transcribe(tmp_audio_file.name, word_timestamps=True)
                # Use the provided transcript for consistency, but Whisper timings
                normalized_transcript = text_normalizations(transcript)

                words_with_timings = []
                if whisper_result and 'segments' in whisper_result:
                    for segment in whisper_result['segments']:
                        if 'words' in segment:
                            for word_info in segment['words']:
                                # Use original word casing/punctuation from Whisper's output if needed,
                                # but strip excess whitespace for consistency.
                                cleaned_word = word_info['word'].strip()
                                if cleaned_word: # Ignore empty strings
                                    words_with_timings.append({
                                        'word': cleaned_word,
                                        'start': float(word_info['start']),
                                        'end': float(word_info['end'])
                                    })
                else:
                    print(f"Whisper did not return segments/words for: {transcript[:50]}...")
                    return None # Indicate failure

                if not words_with_timings:
                    print(f"No word timings extracted by Whisper for: {transcript[:50]}...")
                    return None

                # Prepare data dict for AudioProcessor
                speaker_data_dict = {
                    "audio": {"bytes": audio_bytes},
                    "text": normalized_transcript, # Use the potentially normalized transcript
                    "words": words_with_timings
                }

                # 2. Use AudioProcessor to create the speaker representation
                v3_speaker = self.audio_processor.create_speaker_from_dict(speaker_data_dict)
                return v3_speaker

        except Exception as e:
            print(f"Error during speaker creation (Whisper/AudioProcessor): {e}")
            return None # Indicate failure


    # --- V3 Changes: run method is now a generator ---
    def process_dataset(self, dataset: Dataset):
        """
        Processes a Hugging Face Dataset object in memory and yields training prompts.

        Args:
            dataset (Dataset): The Hugging Face dataset to process.
                               Expected columns: 'text' (str) and 'audio' (dict with 'bytes').

        Yields:
            str: The processed training prompt string for each valid row.
        """
        processed_count = 0
        skipped_count = 0

        # Iterate directly over the dataset
        for i, item in enumerate(tqdm(dataset, desc="Processing Dataset")):
            try:
                # --- Adapt to your dataset's column names ---
                transcript = item.get('text')
                audio_info = item.get('audio')
                # --- End Adapt ---

                if not transcript or not isinstance(transcript, str):
                    print(f"Row {i}: Skipping due to missing or invalid 'text' column.")
                    skipped_count += 1
                    continue

                audio_array = audio_info['array']
                buffer = io.BytesIO()
                # Ensure array is float32 for common compatibility, adjust subtype if needed
                sf.write(buffer, audio_array.astype(np.float32), audio_info['sampling_rate'], format='WAV', subtype='FLOAT')
                buffer.seek(0)
                audio_bytes = buffer.getvalue()

                # Create speaker representation
                speaker = self.create_speaker_representation(audio_bytes, transcript)

                if speaker is None:
                    print(f"Row {i}: Failed to create speaker representation for text: {transcript[:50]}... Skipping.")
                    skipped_count += 1
                    continue

                # Get the V3 training prompt
                prompt = self.prompt_processor.get_training_prompt(speaker)

                processed_count += 1
                yield prompt # Yield the processed prompt string

            except KeyboardInterrupt:
                 print("Processing interrupted by user.")
                 break
            except Exception as e:
                print(f"Row {i}: Unhandled error processing item: {e}", exc_info=True)
                skipped_count += 1
                # Decide if you want to stop on errors or just skip
                continue

        print(f"Dataset processing finished. Processed: {processed_count}, Skipped: {skipped_count}")

if __name__ == "__main__":

    _MODEL_TOKENIZER_PATH = "OuteAI/Llama-OuteTTS-1.0-1B"
    _WHISPER_MODEL = "turbo" # Or "small.en", "medium.en", "large-v2", etc.


    data_processor = DataCreationV3(
        model_tokenizer_path=_MODEL_TOKENIZER_PATH,
        whisper_model_name=_WHISPER_MODEL
    )

    # Process the dataset and collect prompts (or process iteratively)
    all_prompts = []
    print("Starting dataset processing...")
    procced_dataset = data_processor.process_dataset(dataset)
    for prompt in procced_dataset:
        if prompt:
             all_prompts.append({'text': prompt})
    dataset = Dataset.from_list(all_prompts)
    print("Moving Whisper model to CPU")
    data_processor.whisper_model.to('cpu')
    torch.cuda.empty_cache()


Using device: cuda


weights_24khz_1.5kbps_v1.0.pth:   0%|          | 0.00/296M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loading Whisper model: turbo on cuda


100%|█████████████████████████████████████| 1.51G/1.51G [00:22<00:00, 73.4MiB/s]


...

Error during speaker creation (Whisper/AudioProcessor): cuFFT error: CUFFT_INTERNAL_ERROR
Row 1177: Failed to create speaker representation for text: H-huh? N-no, sorry, I-I just kind of spaced out. <... Skipping.


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")

Processing Dataset:  99%|█████████▉| 1181/1195 [23:58<00:16,  1.18s/it]/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")

Processing Dataset:  99%|█████████▉| 1183/1195 [24:00<00:14,  1.18s/it]

Error during speaker creation (Whisper/AudioProcessor): cuFFT error: CUFFT_INTERNAL_ERROR
Row 1182: Failed to create speaker representation for text: It means a lot, and I'm glad you can trust me with... Skipping.


/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")

Processing Dataset:  99%|█████████▉| 1184/1195 [24:01<00:12,  1.14s/it]/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")

Processing Dataset: 100%|█████████▉| 1190/1195 [24:09<00:06,  1.23s/it]/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")

Processing Dataset: 100%|█████████▉| 1192/1195 [24:11<00:03,  1.20s/it]/usr/local/lib/python3.11/dist-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")

Processing Dataset: 100%|█████████▉| 1194/1195 [24:14<00:01,  1.22s/it]/usr/local/lib/python3.1

Dataset processing finished. Processed: 1122, Skipped: 73
Moving Whisper model to CPU


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1122 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,122 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488/1,262,028,800 (1.08% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.144600
2,3.255500
3,3.229200
4,2.975400
5,3.177100
6,3.141400
7,3.252700
8,3.072300
9,3.204900
10,3.195200


In [24]:
input_text = "Hello World! Im a speech generation model using Oute TTS."

In [25]:
#@title Run Inference

import torch
import re
import numpy as np
from typing import Dict, Any
import torchaudio.transforms as T
from transformers import LogitsProcessor
import transformers.generation.utils as generation_utils
from transformers import AutoModelForCausalLM
import re

def get_audio(tokens):
        decoded_output = tokenizer.batch_decode(tokens, skip_special_tokens=False)[0]
        c1 = list(map(int,re.findall(r"<\|c1_(\d+)\|>", decoded_output)))
        c2 = list(map(int,re.findall(r"<\|c2_(\d+)\|>", decoded_output)))

        t = min(len(c1), len(c2))
        c1 = c1[:t]
        c2 = c2[:t]
        output = [c1,c2]
        if not output:
            print("No audio tokens found in the output")
            return None

        return data_processor.audio_processor.audio_codec.decode(
            torch.tensor([output], dtype=torch.int64).to(data_processor.audio_processor.audio_codec.device)
        )

class RepetitionPenaltyLogitsProcessorPatch(LogitsProcessor):
    def __init__(self, penalty: float):
        penalty_last_n = 64
        print("🔄 Using patched RepetitionPenaltyLogitsProcessor -> RepetitionPenaltyLogitsProcessorPatch | penalty_last_n: {penalty_last_n}")
        if penalty_last_n is not None:
            if not isinstance(penalty_last_n, int) or penalty_last_n < 0:
                raise ValueError(f"`penalty_last_n` has to be a non-negative integer, but is {penalty_last_n}")
        if not isinstance(penalty, float) or penalty <= 0:
            raise ValueError(f"`penalty` has to be a positive float, but is {penalty}")

        self.penalty_last_n = penalty_last_n
        self.penalty = penalty

    @torch.no_grad()
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        """
        Args:
            input_ids (`torch.LongTensor`):
                Indices of input sequence tokens in the vocabulary (shape `(batch_size, sequence_length)`).
            scores (`torch.FloatTensor`):
                Prediction scores of a language modeling head (shape `(batch_size, vocab_size)`).

        Returns:
            `torch.FloatTensor`: The modified prediction scores.
        """
        # Check if penalties should be applied
        if self.penalty_last_n == 0 or self.penalty == 1.0:
            return scores

        batch_size, seq_len = input_ids.shape
        vocab_size = scores.shape[-1]

        # Process each batch item independently
        for b in range(batch_size):
            # 1. Determine the penalty window
            start_index = max(0, seq_len - self.penalty_last_n)
            window_indices = input_ids[b, start_index:] # Shape: (window_len,)

            if window_indices.numel() == 0: # Skip if window is empty
                continue

            # 2. Find unique tokens within the window
            tokens_in_window = set(window_indices.tolist())

            # 3. Apply repetition penalty to the scores for this batch item
            for token_id in tokens_in_window:
                if token_id >= vocab_size:
                    continue

                logit = scores[b, token_id]

                if logit <= 0:
                    logit *= self.penalty
                else:
                    logit /= self.penalty

                # Update the score
                scores[b, token_id] = logit

        return scores

generation_utils.RepetitionPenaltyLogitsProcessor = RepetitionPenaltyLogitsProcessorPatch
AutoModelForCausalLM.generate = generation_utils.GenerationMixin.generate

if __name__ == "__main__":
    formated_text = "<|text_start|>"+input_text+"<|text_end|>"
    prompt = "\n".join([
        "<|im_start|>",
        formated_text,
        "<|audio_start|><|global_features_start|>",
    ])

    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    print("Generating token sequence...")
    generated_ids = model.generate(
        **model_inputs,
        temperature=0.4,
        top_k=40,
        top_p=0.9,
        repetition_penalty=1.1,
        min_p=0.05,
        max_new_tokens=2048, # Limit generation length
    )
    print("Token sequence generated.")


    generated_ids_trimmed = generated_ids[:, model_inputs.input_ids.shape[1]:]
    audio = get_audio(generated_ids)
    audio = audio.cpu()
    from IPython.display import Audio, display
    display(Audio(audio.squeeze(0), rate=24000))


Generating token sequence...
🔄 Using patched RepetitionPenaltyLogitsProcessor -> RepetitionPenaltyLogitsProcessorPatch | penalty_last_n: {penalty_last_n}
Token sequence generated.
